<a href="https://colab.research.google.com/github/MasaoMori/Tokyo-Ward-Assembly-Election/blob/main/untitled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd

pd.options.display.float_format = '{:,.0f}'.format

ele_dat = pd.read_excel('./区議会議員選挙過去データ.xlsx', sheet_name="選挙結果")
ele_par = pd.read_excel('./区議会議員選挙過去データ.xlsx', sheet_name="政党名")

ele_result = pd.merge(ele_dat, ele_par, on="党派")

In [19]:
voters_tmp = ele_result[['年','区名','有権者数','投票者数']]
voters_q = voters_tmp[~(voters_tmp.duplicated())]
voters = voters_q.pivot_table(index=['区名','年','有権者数','投票者数'],
                              aggfunc=sum)
voters_q

,年,区名,有権者数,投票者数
0,2011,目黒,213142,"83,040"
11,2015,目黒,219196,"86,254"
22,2019,目黒,228386,"91,994"
33,2011,品川,296585,"120,354"
43,2015,品川,302599,"121,282"
53,2019,品川,322379,"128,049"
63,2011,大田,561414,"243,036"
70,2015,大田,572311,"242,316"
81,2019,大田,599370,"256,171"
97,2011,渋谷,168344,"67,708"


In [30]:
party_name = ele_par[~(ele_par['略称党名'] == '諸派')][['略称党名','党順序']]
#party_name.sort_values('党順序')

import numpy as np

pv = ele_result.pivot_table(index=['略称党名','党順序','年'], columns='区名', aggfunc=np.mean, values=['得票数'])
#pv[['自民','民主','立憲','国民','共産','社民']].applymap('{:,.00f}'.format)
v = pv.loc[['自民','民主','立憲','国民','共産','社民'],:].applymap('{:,.0f}'.format)
vv=v.reset_index(level='党順序',drop=True)
v

得票数                                   
区名                中央     台東     品川     大田     渋谷     目黒
略称党名 党順序 年                                             
自民   1   2011  1,143  1,953  2,448  4,099  1,611  1,704
         2015  1,299  1,804  2,448  4,118  1,517  1,641
         2019  1,430  1,441  2,461  4,018  1,453  1,641
民主   80  2011  1,725  1,633  1,882  2,582  1,261  1,174
         2015  1,143  1,983  1,882  3,787  1,866  1,796
         2019    nan    nan    nan    nan    nan  1,796
立憲   2   2019  1,426  1,840  3,923  5,265  2,439    nan
国民   6   2019    485  1,554  1,601  1,968    603    nan
共産   5   2011    992  1,831  2,472  3,524  1,439  1,681
         2015  1,281  2,021  2,472  4,115  1,769  2,337
         2019  1,032  1,891  2,404  3,375  1,334  2,337
社民   8   2011    nan    418  1,115  1,527    nan  1,050
         2015    nan    652  1,115    nan    nan    950
         2019    nan    nan    nan    nan    nan    950